In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer

In [2]:
%matplotlib inline

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV

In [4]:
train_df=pd.read_csv("Train_split.csv",index_col=[0])
test_df=pd.read_csv("Test_split.csv",index_col=[0])

In [5]:
train_class=train_df['Discretized_Read_Rate']
train_df.drop(['id','read_rate','Discretized_Read_Rate'],axis=1,inplace=True)

In [6]:
test_class=test_df['Discretized_Read_Rate']
test_df.drop(['id','read_rate','Discretized_Read_Rate'],axis=1,inplace=True)

In [7]:
vectorizer = DictVectorizer(sparse=False)

In [8]:
train_X = vectorizer.fit_transform(train_df.to_dict('records'))

In [9]:
test_X = vectorizer.transform(test_df.to_dict('records'))

In [42]:
def customScorer(actualY,predictedY):
    """
    Develop a custom scoring mechanism for model selection. Due to class imbalance, 
    give same importance for all classes. Penalize wrong classification in smaller classes
    more then larger classes. This class creates a score by summing out each row in confusion matrix
    and calculates accuracy.
    """
    cm=confusion_matrix(actualY,predictedY)
    score=1
    for en,row in enumerate(cm):
        score+=row[en]/np.sum(row)
    return(score)

In [11]:
cscore=make_scorer(customScorer,greater_is_better=True)

In [44]:
param_grid = [
    {'C': [1], 'kernel': ['linear']},
#     {'C': [0.01,0.1,0.5,1,5,10], 'gamma': [0.001, 0.0001,0.01,0.1,0.5,1,5], 'kernel': ['rbf']},
#     {'C': [0.01,0.1,0.5,1,5,10], 'coef0': [0.001, 0.0001,0.01,0.1,0.5,1,5,10,20], 'degree':[5,6,7],'kernel': ['poly']},
#     {'C': [0.01,0.1,0.5,1,5,10], 'coef0': [0.001, 0.0001,0.01,0.1,0.5,1,5],'kernel': ['sigmoid']}
    ]
svc=SVC(class_weight='balanced',C=5,kernel='rbf',gamma=0.01)
# clf = GridSearchCV(estimator=svc, param_grid=param_grid,n_jobs=1,scoring=cscore)
clf=svc

In [45]:
clf.fit(train_X,train_class)

SVC(C=5, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [46]:
predictions=clf.predict(test_X)

In [47]:
pd.DataFrame(confusion_matrix(test_class,predictions))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1240,465,324,153,165,101,94,62,24,36,23,24,20,16
1,78,131,48,46,20,46,6,24,15,19,14,4,10,1
2,37,49,58,29,52,27,18,9,7,9,7,9,3,1
3,23,31,15,23,7,13,15,9,11,4,6,3,5,1
4,12,13,15,9,26,11,7,2,3,9,5,6,1,3
5,3,8,4,13,5,15,5,5,7,4,6,2,1,1
6,3,4,8,6,3,3,9,3,4,7,2,7,3,1
7,0,4,0,3,2,10,5,4,7,6,3,1,1,1
8,2,0,1,2,3,4,2,3,2,3,2,0,1,1
9,3,1,1,0,0,0,2,4,2,4,0,3,3,0


In [48]:
cm=confusion_matrix(test_class,predictions)

In [49]:
customScorer(test_class,predictions)

4.1923075747756968

In [50]:
np.sum(np.diag(cm))/np.sum(cm)

0.37623762376237624